In [2]:
%load_ext rpy2.ipython

In [4]:
%%R
source("http://bioconductor.org/biocLite.R")
biocLite("phyloseq")

Bioconductor version 3.0 (BiocInstaller 1.16.5), ?biocLite for help
A new version of Bioconductor is available after installing the most recent
  version of R; see http://bioconductor.org/install
BioC_mirror: http://bioconductor.org
Using Bioconductor version 3.0 (BiocInstaller 1.16.5), R version 3.1.2.
Installing package(s) 'phyloseq'
also installing the dependencies ‘base64enc’, ‘brew’, ‘checkmate’, ‘DBI’, ‘fail’, ‘RSQLite’, ‘sendmailR’, ‘XML’, ‘xtable’, ‘GenomeInfoDb’, ‘BatchJobs’, ‘BBmisc’, ‘AnnotationDbi’, ‘annotate’, ‘Formula’, ‘latticeExtra’, ‘acepack’, ‘gridExtra’, ‘pkgmaker’, ‘registry’, ‘rngtools’, ‘gridBase’, ‘doParallel’, ‘RJSONIO’, ‘chron’, ‘GenomicRanges’, ‘RcppArmadillo’, ‘Biobase’, ‘BiocParallel’, ‘genefilter’, ‘locfit’, ‘geneplotter’, ‘Hmisc’, ‘iterators’, ‘NMF’, ‘irlba’, ‘permute’, ‘ade4’, ‘ape’, ‘biom’, ‘data.table’, ‘DESeq2’, ‘foreach’, ‘igraph’, ‘multtest’, ‘vegan’

trying URL 'http://cran.fhcrc.org/src/contrib/base64enc_0.1-2.tar.gz'
Content type 'application/x-gz

In [ ]:
%%R
library(phyloseq)
library(dplyr); library(tidyr)
library(ggplot2)

In [ ]:
%%R
physeq = import_biom("../../SeqData/otu_table.tax.meta.biom", parseFunction = parse_taxonomy_greengenes)

In [ ]:
%%R
mdf = psmelt(physeq)

In [ ]:
%%R -w 900
d.plot = mdf %>%
    group_by(Sample) %>%
    mutate(Abundance = Abundance / sum(Abundance)) %>%
    group_by(Sample, Phylum, Month, Trtmt) %>%
    summarize(Abundance = sum(Abundance)) %>%
    arrange(Month) %>%
    

#scheme <- iwanthue()
#colors = scheme$hex(length(unique(d.plot$Rank2)), 
#                    color_space = hcl_presets$intense)

p = ggplot(d.plot, aes(x = Sample, y = Abundance, fill = Rank2))

p = p + facet_grid(. ~ Trtmt, scales = "free_x", space = "free")

p = p + geom_bar(stat = "identity", width = 0.70)

p = p + scale_fill_manual(values = colors)

p = p + guides(fill = guide_legend(ncol = 2))

p = p + theme_bw()

p = p + theme(axis.text.x = element_blank())

p

In [ ]:
%%R
# Normalizes the sample counts by the total - i.e., reporting what fraction of each sample each OTU makes up.
physeq_snorm = transform_sample_counts(physeq, function(x) x / sum(x))

In [ ]:
%%R
# Computing the weighted unifrac distance matrix and NMDS in parallel
registerDoParallel(cores=4)
ord.NMDS.full = ordinate(physeq_snorm, method = "NMDS", distance = "Bray", fast = TRUE, parallel = TRUE)
NMDS.full = plot_ordination(physeq_snorm, ord.NMDS.full, axes = c(1,2), justDF = TRUE)
NMDS.full$Samples = rownames(NMDS.full)

In [ ]:
%%R -h 300 -w 900
D <- NMDS.full
# Sets the relevant ordination

colnames(D)[1:2]<-c("NMDS1","NMDS2")
# Renames the axis columns

#D$AmdmtName = ifelse(D$Amdmt==1,"PyOM",ifelse(D$Amdmt==2,"Stover","Soil only"))
#D$AmdmtName = factor(D$AmdmtName, levels=c("Soil only", "PyOM", "Stover"))
#D$DayName = ifelse(D$Day==1,"Day 12",ifelse(D$Day==2,"Day 82","Day 1"))

# Creates an plot object, using the df.sites data, with the x and y axes specified as the PCoA axes,
p = ggplot(D, aes(x = NMDS1, y = NMDS2, color = Trtmt, shape = Month))

# Determines the size of the symbols
p = p  + geom_point(size=3.5) + scale_colour_manual(values=c("orange","gold1","red3"))

p = p + guides(colour = guide_legend(""), shape = guide_legend(""))

# Determines the color theme of the plot overall
p = p + theme_bw() +
  theme(
    plot.background = element_blank()
   ,panel.grid.major = element_blank()
   ,panel.grid.minor = element_blank()
  ) +
  theme(strip.text.x = element_text(size=14, face="bold"),axis.text = element_text(size=12),legend.text = element_text(size=12),
          strip.background = element_rect(colour="white", fill="white"))

# Displays the plot object
p = p + facet_wrap(~DayName, scales="free")
# The axis scales could be all optimized individually
#p = p + facet_wrap(~DayName)
# Adds a "facet_wrap" to it, which means it will plot it separately by whatever we specify - here, Days
plot.full.byDay = p
p